Nil Mazouzi More 1674092 <br>
Jan Sans Domingo 1673276

In [12]:
import numpy as np
import pandas as pd
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, average_precision_score, roc_curve, precision_recall_curve, classification_report, make_scorer, auc
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold

Abans de començar a fer res, anem a veure com és la base de dades del problema.

La web del Kaggle ens  descriu els nostres atributs, i nosaltres els classifiquem segons aquesta descripció:
| Atribut                     | Tipus              | Descripció                                                                       |
|-----------------------------|--------------------|----------------------------------------------------------------------------------|
| class                       | Binari (Target)    | classes: edible=e, poisonous=p                                                  |
| cap-shape                   | Categòric          | cap-shape: bell=b, conical=c, convex=x, flat=f, knobbed=k, sunken=s             |
| cap-surface                 | Categòric          | cap-surface: fibrous=f, grooves=g, scaly=y, smooth=s                            |
| cap-color                   | Categòric          | cap-color: brown=n, buff=b, cinnamon=c, gray=g, green=r, pink=p, purple=u, red=e, white=w, yellow=y |
| bruises                     | Binari             | bruises: bruises=t, no=f                                                        |
| odor                        | Categòric          | odor: almond=a, anise=l, creosote=c, fishy=y, foul=f, musty=m, none=n, pungent=p, spicy=s |
| gill-attachment             | Categòric          | gill-attachment: attached=a, descending=d, free=f, notched=n                    |
| gill-spacing                | Categòric          | gill-spacing: close=c, crowded=w, distant=d                                     |
| gill-size                   | Binari             | gill-size: broad=b, narrow=n                                                    |
| gill-color                  | Categòric          | gill-color: black=k, brown=n, buff=b, chocolate=h, gray=g, green=r, orange=o, pink=p, purple=u, red=e, white=w, yellow=y |
| stalk-shape                 | Binari             | stalk-shape: enlarging=e, tapering=t                                            |
| stalk-root                  | Categòric          | stalk-root: bulbous=b, club=c, cup=u, equal=e, rhizomorphs=z, rooted=r, missing=? |
| stalk-surface-above-ring    | Categòric          | stalk-surface-above-ring: fibrous=f, scaly=y, silky=k, smooth=s                 |
| stalk-surface-below-ring    | Categòric          | stalk-surface-below-ring: fibrous=f, scaly=y, silky=k, smooth=s                 |
| stalk-color-above-ring      | Categòric          | stalk-color-above-ring: brown=n, buff=b, cinnamon=c, gray=g, orange=o, pink=p, red=e, white=w, yellow=y |
| stalk-color-below-ring      | Categòric          | stalk-color-below-ring: brown=n, buff=b, cinnamon=c, gray=g, orange=o, pink=p, red=e, white=w, yellow=y |
| veil-type                   | Categòric          | veil-type: partial=p, universal=u                                               |
| veil-color                  | Categòric          | veil-color: brown=n, orange=o, white=w, yellow=y                                |
| ring-number                 | Categòric          | ring-number: none=n, one=o, two=t                                               |
| ring-type                   | Categòric          | ring-type: cobwebby=c, evanescent=e, flaring=f, large=l, none=n, pendant=p, sheathing=s, zone=z |
| spore-print-color           | Categòric          | spore-print-color: black=k, brown=n, buff=b, chocolate=h, green=r, orange=o, purple=u, white=w, yellow=y |
| population                  | Categòric          | population: abundant=a, clustered=c, numerous=n, scattered=s, several=v, solitary=y |
| habitat                     | Categòric          | habitat: grasses=g, leaves=l, meadows=m, paths=p, urban=u, waste=w, woods=d     |

Veiem que el nostre target és class, que és un atribut binari que representa si el bolet és comestible o n (e = comestible, p = no comestible)

Si ens fixem, podem observar que a la descripció de la web del Kaggle ens diu que a l'atribut "stalk-root" els missing values o nans, es representen amb un interrogant.

Com el dataset no té variables numèriques, a priori no podem fer veure la correlació. 

En quant a la distribució, la pàgina del Kaggle ens proporciona la següent informació:
| Atribut           | Categories/Valors | Distribució                              | Valid (%) | Mismatched (%) | Missing (%) | Unique | Most Common |
|--------------------|-------------------|------------------------------------------|-----------|----------------|-------------|--------|-------------|
| class             | edible=e, poisonous=p | e: 52%, p: 48%                          | 100%      | 0%             | 0%          | 2      | e (52%)     |
| cap-shape         | bell=b, conical=c, convex=x, flat=f, knobbed=k, sunken=s | x: 45%, f: 39%, Other: 16%              | 100%      | 0%             | 0%          | 6      | x (45%)     |
| cap-surface       | fibrous=f, grooves=g, scaly=y, smooth=s | y: 40%, s: 31%, Other: 29%              | 100%      | 0%             | 0%          | 4      | y (40%)     |
| cap-color         | brown=n, buff=b, cinnamon=c, gray=g, green=r, pink=p, purple=u, red=e, white=w, yellow=y | n: 28%, g: 23%, Other: 49% | 100%      | 0%             | 0%          | 10     | n (28%)     |
| bruises           | bruises=t, no=f   | [No valid data provided]                 | 0%        | 100%           | 0%          | 0      | N/A         |
| odor              | almond=a, anise=l, creosote=c, fishy=y, foul=f, musty=m, none=n, pungent=p, spicy=s | n: 43%, f: 27%, Other: 30% | 100%      | 0%             | 0%          | 9      | n (43%)     |
| gill-attachment   | attached=a, descending=d, free=f, notched=n | [No valid data provided]                 | 0%        | 100%           | 0%          | 0      | N/A         |
| gill-spacing      | close=c, crowded=w, distant=d | c: 84%, w: 16%                          | 100%      | 0%             | 0%          | 2      | c (84%)     |
| gill-size         | broad=b, narrow=n | b: 69%, n: 31%                          | 100%      | 0%             | 0%          | 2      | b (69%)     |
| gill-color        | black=k, brown=n, buff=b, chocolate=h, gray=g, green=r, orange=o, pink=p, purple=u, red=e, white=w, yellow=y | b: 21%, p: 18%, Other: 60% | 100% | 0%             | 0%          | 12     | b (21%)     |

Podem veure que el target, que és l'atribut class està prou balancejat, lo qual significa que els models podran classificar efectivament "class" sense necessitar de reajustar aquest atribut. Això pot significar que podrem utilzar l'accuracy com a mesura. No podriem fer-ho si estigues desbalancejat (p.e. 85% e, 15% p), ja que si el model classifiqués tots els bolets com a edible tindria una accuracy del 85% i pot ser molt perillòs, ja que classificar un bolet com a comestible quan en realitat és verinós pot tenir series conseqüències.





In [13]:
import pandas as pd

file_path = 'mushrooms.csv'
df = pd.read_csv(file_path)

df.head()


,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


Com podem veure, el dataset té 23 atributs: class ,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population i habitat.

Entre aquests atributs no en trobem cap continu, tots són binaris o categòrics.

In [17]:
def print_nans(df):
    nan_percentages = (df == '?').mean() * 100
    for col, percentage in nan_percentages.items():
        print(f"{col}: {percentage:.2f}%")

print_nans(df)


class: 0.00%
cap-shape: 0.00%
cap-surface: 0.00%
cap-color: 0.00%
bruises: 0.00%
odor: 0.00%
gill-attachment: 0.00%
gill-spacing: 0.00%
gill-size: 0.00%
gill-color: 0.00%
stalk-shape: 0.00%
stalk-root: 30.53%
stalk-surface-above-ring: 0.00%
stalk-surface-below-ring: 0.00%
stalk-color-above-ring: 0.00%
stalk-color-below-ring: 0.00%
veil-type: 0.00%
veil-color: 0.00%
ring-number: 0.00%
ring-type: 0.00%
spore-print-color: 0.00%
population: 0.00%
habitat: 0.00%


Com podem veure, només trobem valors nans a stalk-root. 

In [15]:
def missing_value_class_distribution(df, feature, target):
    missing_rows = df[df[feature] == '?']    
    class_distribution = missing_rows[target].value_counts(normalize=True) * 100
    return class_distribution.to_dict()

result = missing_value_class_distribution(df, 'stalk-root', 'class')
print(result)

# Distribución de stalk-root por clase
# Filtrar los datos donde 'stalk-root' no sea '?'
filtered_df = df[df['stalk-root'] != '?']

# Agrupar por 'stalk-root' y 'class', y contar las ocurrencias
grouped_counts = filtered_df.groupby(['stalk-root', 'class']).size()

# Normalizar dividiendo por el total de cada clase
normalized_distribution = grouped_counts.groupby(level=1).apply(lambda x: x / x.sum())

# Mostrar los resultados
# Reorganizar el DataFrame para una visualización más clara
distribution_table = normalized_distribution.unstack(fill_value=0)

# Mostrar la tabla final
print(distribution_table)





{'p': 70.96774193548387, 'e': 29.03225806451613}
class                    e         p
class stalk-root                    
e     b           0.550459  0.000000
      c           0.146789  0.000000
      e           0.247706  0.000000
      r           0.055046  0.000000
p     b           0.000000  0.860853
      c           0.000000  0.020408
      e           0.000000  0.118738


In [18]:
import numpy as np

def imputar_valores_probabilisticamente(df, feature, target, prob_table):
    """
    Imputa valores faltantes en una columna categórica basada en probabilidades condicionales.
    
    Args:
        df (pd.DataFrame): El DataFrame con los datos.
        feature (str): Nombre de la columna con valores faltantes.
        target (str): Nombre del target (clase objetivo).
        prob_table (pd.DataFrame): Tabla de probabilidades condicionales por clase.
        
    Returns:
        pd.DataFrame: El DataFrame con valores imputados.
    """
    # Crear una copia para no modificar el DataFrame original
    df_copy = df.copy()
    
    # Filtrar las filas con valores faltantes
    missing_indices = df_copy[df_copy[feature] == '?'].index
    
    # Iterar sobre las filas faltantes
    for idx in missing_indices:
        # Obtener la clase correspondiente de la fila
        target_class = df_copy.loc[idx, target]
        
        # Obtener las probabilidades condicionales para la clase
        probabilities = prob_table[target_class]
        
        # Generar un valor según la distribución de probabilidad
        imputed_value = np.random.choice(probabilities.index, p=probabilities.values)
        
        # Imputar el valor generado
        df_copy.at[idx, feature] = imputed_value
    
    return df_copy

# Generar la tabla de probabilidades condicionales (ya calculada por ti)
distribution_table = filtered_df.groupby(['stalk-root', 'class']).size()
distribution_table = distribution_table.groupby(level=1).apply(lambda x: x / x.sum()).unstack(fill_value=0)

# Aplicar la imputación al DataFrame original
df_imputado = imputar_valores_probabilisticamente(df, 'stalk-root', 'class', distribution_table)

# Verificar los resultados
print(df_imputado['stalk-root'].value_counts())


def print_nans(df):
    nan_percentages = (df == '?').mean() * 100
    for col, percentage in nan_percentages.items():
        print(f"{col}: {percentage:.2f}%")

print_nans(df_imputado)

stalk-root
b         3776
(p, b)    1518
e         1120
c          556
(e, b)     413
(p, e)     204
r          192
(e, e)     172
(e, c)      98
(p, c)      38
(e, r)      37
Name: count, dtype: int64
class: 0.00%
cap-shape: 0.00%
cap-surface: 0.00%
cap-color: 0.00%
bruises: 0.00%
odor: 0.00%
gill-attachment: 0.00%
gill-spacing: 0.00%
gill-size: 0.00%
gill-color: 0.00%
stalk-shape: 0.00%
stalk-root: 0.00%
stalk-surface-above-ring: 0.00%
stalk-surface-below-ring: 0.00%
stalk-color-above-ring: 0.00%
stalk-color-below-ring: 0.00%
veil-type: 0.00%
veil-color: 0.00%
ring-number: 0.00%
ring-type: 0.00%
spore-print-color: 0.00%
population: 0.00%
habitat: 0.00%
